In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
# Step 1: Get CALDAY (+7 hours offset)
v_calday = spark.sql("SELECT DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)").collect()[0][0]

v_last_wed_this_month = spark.sql(f"""
SELECT MAX(CAST(datee AS DATE)) AS last_wed_this_month
FROM (
    SELECT explode(sequence(
        date_trunc('month', DATE('{v_calday}')),
        last_day(DATE('{v_calday}')),
        interval 1 day
    )) AS datee
)
WHERE dayofweek(datee) = 4
""").collect()[0][0]

v_last_wed_next_month = spark.sql(f"""
SELECT MAX(CAST(datee AS DATE)) AS last_wed_next_month
FROM (
    SELECT explode(sequence(
        date_trunc('month', add_months(DATE('{v_calday}'), 1)),
        last_day(add_months(DATE('{v_calday}'), 1)),
        interval 1 day
    )) AS datee
)
WHERE dayofweek(datee) = 4
""").collect()[0][0]


v_available_to= spark.sql(f"""SELECT IF(DATE('{v_last_wed_this_month}') >= DATE_ADD( DATE('{v_calday}'),  30 ), DATE('{v_last_wed_this_month}'), DATE('{v_last_wed_next_month}'))""").collect()[0][0]


In [0]:
day_of_week = spark.sql(f"SELECT dayofweek(DATE('{v_calday}'))").collect()[0][0]
if day_of_week==5:
    spark.sql(f"""
        CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_fresh.d_store_group_nso
        --CLUSTER BY store_id
        --OPTIONS (DESCRIPTION = "BẢNG SUGGEST TÍNH PHÂN LOẠI CỬA HÀNG THEO SỨC BÁN BÌNH QUÂN NGÀY") 
        AS
        WITH main AS (
            SELECT
                 DATE('{v_calday}') AS calday,
                ds.business_unit,
                ds.store_id,
                ds.store_name,
                ds.mien AS region,
                CAST(NULL AS DOUBLE) AS base_sale_qty_avg,
                CAST(NULL AS DOUBLE) AS sale_kg,
                'N7' AS store_group_id,
                'NO' AS is_adj,
                CASE
                    WHEN ds.business_unit = '1500' THEN 'Supermarket'
                    WHEN ds.business_unit = '2000' AND ds.concept IN ('WiN', 'Rural') THEN ds.concept
                    ELSE 'Urban'
                END AS concept,
                ds.qlkv,
                ds.email_qlkv,
                ds.is_win_plus,
                ds.email_gdv,
                ds.city,
                ds.region_description AS province,
                'YES' AS is_nso,
                1 AS priority_order,
                 DATE('{v_calday}') AS available_from,  
                 DATE('{v_available_to}') AS available_to 
            FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store ds
            WHERE ds.store_status_desc IN ('Đang hoạt động', 'Sắp đóng cửa')
            AND ds.business_unit = '2000'
            AND ds.is_win_plus != 'YES'
            AND SUBSTR(ds.store_id, 1, 2) <> 'GH'
            AND ds.store_open_date IS NOT NULL
            AND SUBSTR(ds.store_name,1,4) <> 'FWMP'
            AND NOT EXISTS (SELECT 1 FROM {catalog_name}.udp_wcm_gold_db_winare_fresh.d_store_exclude_v2 WHERE UPPER(store_id) = UPPER(ds.store_id))
            AND NOT EXISTS (SELECT 1 FROM {catalog_name}.udp_wcm_gold_db_winare_fresh.d_store_group_temp WHERE UPPER(store_id) = UPPER(ds.store_id))

        )
        SELECT * FROM main;
    """)